In [ ]:
print("hello")

In [ ]:
import random
import math
from scipy.special import erfinv
import matplotlib.pyplot as plt
import numpy as np


class Measurement:
    def __init__(self, mean: float, std: float) -> None:
        self.mean = mean
        self.std = std
        self.rng = random.Random()

    def measure(self):
        # return a normally distributed measurement
        area = self.rng.random()
        x = self.std * math.sqrt(2) * erfinv(2 * area - 1) + self.mean
        return x

    def measure_n(self, n: int):
        return [
            self.measure() for _ in range(max(n, 0))
        ]


class MultipleMeasurements(Measurement):
    def __init__(self, mean: float, std: float, measure_std: float) -> None:
        super().__init__(mean, std)
        self.measure_std = measure_std

    def measure(self):
        # broad uncertainty
        x = super().measure()

        # measurement uncertainty
        area = self.rng.random()
        measured_x = x + self.measure_std * math.sqrt(2) * erfinv(2 * area - 1)
        return measured_x


# plotting
def plot_measurements(measure: Measurement, N, s):
    measurements = []
    halfranges = []
    stds = []
    for _ in range(N):
        measurement = measure.measure_n(s)
        halfrange = (max(measurement) - min(measurement)) / 2
        std = np.std(measurement)

        measurements.extend(measurement)
        halfranges.append(halfrange)
        stds.append(std)

    # plot
    plt.figure(figsize=(23, 6))
    plt.subplot(1, 3, 1)
    plt.title("Measurements Distribution")
    plt.hist(measurements, bins=50)

    plt.subplot(1, 3, 2)
    plt.title("Halfranges Distribution")
    plt.hist(halfranges, bins=50)

    plt.subplot(1, 3, 3)
    plt.title("Std Distribution")
    plt.hist(stds, bins=50)

    plt.show()

    return np.std(measurements), np.mean(halfranges), np.mean(stds)



# average = 30.6
average = 10.2
std = 2
m_std = 0.1
N = 5000
s = 4

measure = MultipleMeasurements(average, std, m_std)
true_std, halfranges, stds = plot_measurements(measure, N, s)

print(f"with data ({average}, {std}), measurement (x, {m_std})")
print(f"with {N} trials, sample size {s}")

print('')
print('uncertainties')
print(f"true std            : {true_std:.2f}")
print(f"half ranges average : {halfranges:.2f}")
print(f"stds average        : {stds:.2f}")
print(f"titre measurement   : {m_std:.2f}")





In [ ]:
# try to plot halfrange average against sample size

def normalizer(s):
    a, b = 5.8223816945, -9.4638559351  
    return math.log((abs(a * s + b)))/2.5

def plot_halfrange_against_samplesize(measure: Measurement, N, s_max):
    ranges = list(range(2, s_max))
    true_stds = []
    halfrange_stds = []
    stats_stds = []
    for s in ranges:
        measurements = []
        halfranges = []
        statsstds = []
        for _ in range(N):
            measurement = measure.measure_n(s)
            halfrange = (max(measurement) - min(measurement)) / 2
            stats_std = np.std(measurement) * s / (s-1)
            measurements.extend(measurement)
            halfranges.append(halfrange)
            statsstds.append(stats_std)

        true_stds.append(np.std(measurements))
        halfrange_stds.append(np.mean(halfranges))
        stats_stds.append(np.mean(statsstds))

    normalized_halfrange = [
        r / normalizer(s+1) for s, r in enumerate(halfrange_stds)
    ]
   
    # plot
    plt.figure(figsize=(10, 6))
    plt.title("Estimated Standard Deviations against Sample Size")
    plt.ylabel("Estimated Std")
    plt.xlabel("Sample Size")
    plt.scatter(ranges, true_stds, label="true std")
    plt.scatter(ranges, halfrange_stds, label="half range std")
    plt.scatter(ranges, stats_stds, label="stats normalization")
    plt.scatter(ranges, normalized_halfrange, label="half range normalization")

    plt.legend()
    plt.show()

    return (ranges, halfrange_stds)


average = 36.2
std = 3.2
m_std = 5.2
N = 2000
s_max = 30

measure = MultipleMeasurements(average, std, m_std)

print(f"with data ({average}, {std}), measurement (x, {m_std})")
print(f"with {N} trials")
ranges, halfranges = plot_halfrange_against_samplesize(measure, N, s_max)

# export to csv
def export_ranges(ranges, halfranges):
    import csv
    with open('halfrange.csv', 'w', newline='') as file:
        writer = csv.writer(file, delimiter=',')
        writer.writerow(['samplesize', 'averagehalfrange'])
        for range, halfrange in zip(ranges, halfranges):
            writer.writerow([range, halfrange])

# export_ranges(ranges, halfranges)




In [ ]:
titres = [
    30.8,
    30.6,
    30.4,
    30.5,
    30.2,
]

mean = np.mean(titres)
std = (max(titres) - min(titres)) / 2
print(mean, std, std / normalizer(3), np.std(titres) * 4 / 3)